In [1]:
from pymongo import MongoClient
from gensim.models.wrappers.ldamallet import malletmodel2ldamodel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_documents
from sklearn.pipeline import Pipeline
import pyLDAvis.gensim
import time
import os
import random

from preprocessor import *


X:\Programs\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
X:\Programs\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
X:\Programs\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Conectando ao MongoDB

In [2]:

client = MongoClient('localhost', 27017)
db = client["discursoDB"]
discursos = db["discursos"]

In [14]:
start_time = time.time()

# all_discursos = discursos.find()

# discursos_list = random.sample(list(all_discursos), 5000)

# discursos_list = discursos.find(
#     {"$and": [
#         {"SiglaPartidoParlamentarNaData": {"$eq": "PT"}},
#         {"Conteudo": {
#             "$ne": ""
#         }}
#     ]})

# discursos_list = random.sample(list(discursos_list), 1)

discursos_list = discursos.find({"Conteudo": {
            "$ne": ""
        }})

text = []

for disc in discursos_list:
    text.append(disc["Conteudo"])

pipe = Pipeline([
    ('cleaning', Cleaner()),
    ('stopwords', StopWords(lang='portuguese', tokenize=True)),
    ('stemming', Stemmer(lang='portuguese', fit_reuse=True))])
pipe.fit(text)
res = pipe.transform(text)

# preproc = Preprocessor(max_word_lenght=2)
# preproc.fit(text)
# text = preproc.transform(text)

elapsed_time = time.time() - start_time
print(time.strftime("Discursos preprocessados, demorou %H:%M:%S:%m",
                    time.gmtime(elapsed_time)))


Discursos preprocessados, demorou 00:29:52:01


In [16]:
with open("discursos_all.txt", 'w+') as f:
    f.write(str(res))

In [21]:
f = open("discursos_all.txt","r")
discursos_file = f.read()
f.close()

y = eval(discursos_file)

print(y[0])

abdi nasciment blocopdtrj lid pronunc seguint discurs president sras srs senador proposit assum hoj tribun inform presidenc dest cas coleg senador realiz ontem reunia final comissa julgament premi cruz sous honr particip lad senador esperidia amin deput paul gouv poet gerard mell moura professor iaponan soar trabalh apresent numer consider excelent funca pouc temp dispunh candidat oit hav sid elimin imediat raza hav autor infring regul artig estabelec carat individual cad obra referent necess identificaca pseudonim relat datalimit entreg trabalh monograf restant enta distribu decisa comissa dois relator poet gerard mell moura responsavel examin trabalh categor geral professor iaponan soar fic categor estud ambos apresent ontem relatori unanim aprov comissa diss result escolh vencedor assim categor estud fic primeir lug monograf intitul cruz sous sol negr autor carl albert shimot martins jair sant ampar cruz sous biograf autor unic trabalh merec menca honros ness categor luis claudi rib

In [28]:
y[8]

''

In [5]:
res[0]
# " ".join(pipe.predict([res[0]])[0])


'aloizi mercad blocopt revisa orador president seman pass sub tribun fiz pronunc long sobr cris sen cerc tres hor mei receb vint apart senador pratic tod legend entant nao consegu pesso fiz entend men profund sentiment realment estav busc constru enfrent resolv cris sen tamb dialog president sarney duas circunstanc inclusiv pratic tod banc dialog president luiz inaci lul silv objet tamb nest cas interpretaco divers muit dir absolut improcedent diant quer express posica banc atrav not escrit formaliz bastant objet not oficial banc sen senador part trabalh comunic lideranc moment algum discussa sobr cris sen exig integr abdic posico individu tampouc fez president republ luiz inaci lul silv durant encontr palaci alvor seman pass portant banc sent respeit president lul autonom parlament sen reafirm banc senador long tod discussa sobr cris sen mantev posica suger gest grandez garant credibil investigaco senador jos sarney licenc temporari sen pud aprofund investigaco constru propost soluca 

In [7]:

start_time = time.time()
# Create a corpus from a list of texts
data = [a.split() for a in res]

dictionary = Dictionary(data)

corpus = [dictionary.doc2bow(t) for t in data]


In [9]:

mallet_path = 'X:\\Programs\\mallet\\mallet-2.0.8\\bin\\mallet.bat'

In [12]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [13]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=res, start=2, limit=40, step=6)

NameError: name 'CoherenceModel' is not defined

In [ ]:

# os.environ['MALLET_HOME'] = 'X:\\Programs\\mallet\\mallet-2.0.8\\'
mallet_path = 'X:\\Programs\\mallet\\mallet-2.0.8\\bin\\mallet.bat'

# Train the model on the corpus.
lda = LdaMallet(mallet_path, corpus, id2word=dictionary, num_topics=20)
# lda = LdaModel(corpus, id2word=dictionary, num_topics=10, alpha='auto', eval_every=5, chunksize=10, passes=10, decay=0.9)

elapsed_time = time.time() - start_time
print(time.strftime("Lda model criado, demorou %H:%M:%S:%m", time.gmtime(elapsed_time)))


In [0]:
for index, topic in lda.show_topics(formatted=False, num_words=10):
    print('Topic: {} \nWords: {}'.format(
        index, [pipe.predict([w[0]])[0][0] for w in topic]))
        # index, [pipe.predict([w[0]])[0][0] for w in topic]))